## Summary

 - open-source framework for desktop apps built using web technology (TypeScript, HTML, CSS)
 - built on Chromium and NodeJS (statically built into shipped binaries)
 - developed by OpenJS Foundation
 - VSCode is built on it

## Dependencies

 - `npm install electron --save-dev`
 - `npm install typescript @types/node --save-dev` only if using `TypeScript`

## Entry Point

 - the `main` field in `package.json` tells many frameworks, including Electron, where to start from
 - when you run `electron .`, electron will search the current folder for `main` from `package.json` and run it as a GUI app
   - if the script does nothing, it will have nothing to show
 - default if not configured is `index.js`
 - create an __npm script__ (eg. `start`) to run `electron .`

## Program Structure

 - global `app` object from `electron` package can be used to hook into the lifecycle
 - `app.whenReady()` returns a promise to run when the app is ready to run (good place to start)
 - `BrowserWindow` from `electron` package is object you can create and then load HTML files with
 - use `app.on()` to listen for other events
 - use `app.quit()` to end the program
 - overeall, the `main process` is like the browser (which you can extend and customize as a desktop app), and it loads HTML files, which are like the static HTML files you usually ignore in React and Angular apps (though you don't have to)
   - you can run bundles from React, etc. from these HTML files
 
 ## Lifecycle
 
 - `app.whenReady()` as described above
 - `app.on('window-all-closed')` for when all windows of the app are closed
 - `app.on('activate')` for when the app is focused (even if no windows)
 
 ## BrowserWindow
 
 - `loadFile(filename)` to load an HTML file
 - `getAllWindows()` to get the open windows as an array (check length if need number)
 
## System-Specific Behavior

 - use Node's `process.platform` to determine which system(s) you are on
  - eg. `darwin` if __MacOS__
 - when __all windows are closed__:
   - on Windows and Linux, close the app with `app.quit()`
   - on MacOS, create a new window on app activation
     - you can rely on `activate` not getting called for the other platforms if exit right away
     
## DOM

 - the __main process__ (entry point script) has no access to the DOM
   - because the DOM is in a __separate process__ called the __renderer__
 - to run code in the DOM of an HTML page before it renders, do the following:
   1. Create a JS file to be run in that context standalone
   1. In that script, subscribe to an event like `DOMContentLoaded`
   1. Compute the absolute path to the script using Node as in the __quickstart tutorial__
      - relies on some imports from `path` and `url` node libraries
   1. Add `webPreferences.preload` field to the 3rd param of your `BrowserWindow` constructor
 - to run any script code you want in the DOM, do the following:
   - add a `<script>` tag in the HTML file pointing to a JS bundle
   - build that JS bundle with Webpack, etc.
     - then you can use whatever technology you want, such as React, just like with normal web apps

## DevTools

???

## Scaffolding/Templates

???

## Distribution

Electron apps can be packaged and distributed using `Electron Forge`

TBD ???

## Using TypeScript

To convert a default JS electron app to TypeScript:

1. Install the typescript dependencies to the project (as shown in Dependencies above)
1. Create your `tsconfig.json` file to tell tsc what to do.  For instance:
    ```JS
    {
      "compilerOptions": {
        "target": "ESNext",
        "module": "es2020",
        "strict": true,
        "esModuleInterop": true,
        "outDir": "./dist",
        "rootDir": "./src",
        "moduleResolution": "node",
        "resolveJsonModule": true,
        "skipLibCheck": true,
        "typeRoots": ["node_modules/@types"]
      },
      "exclude": ["node_modules"]
    }

    ```
   - Note that we told it to look for a `src` folder and output to a `dist` folder
1. Move your JS files into the `src` folder and change the extension to `.ts` (plus any code changes inside)
1. In `package.json`, change `main` to `dist/main.js`, add a `build` script that calls `tsc`, and add `npm run build &&` to the front of the `start` script (for instance)
1. Any place that references JS files will need to reference them from the appropriate new place
   - usually `dist/thefile.js` to get the build outputs from `tsc`
   - but the code to get the absolute path of the current script will automatically know it's in `dist` because that's where the main JS is running from

## Using React, Angular, etc.

Differences:
1. The __main process__ will still be in TS or JS and still needs to load a main HTML file
1. The main HTML file still needs to include the bundled script, just like always
1. You handle the react building/bundling (eg. via WebPack) in your `package.json` scripts as usual

It works basically like a normal statically built app except you have an additional script that loads the top-level HTML, which references JS files that should have been made by the build process before electron runs.

NOTE: that means your React or Angular app could work as a web app as well as the electron app.  You just won't have any of the stuff that runs in the main process.

To build from scratch:

1. Start by creating an Angular or React app for the in-browser part as normal for the web framework you're using
1. Install `electron` to the project and add/modify `package.json` scripts to use electron to run a new top-level TS file (`main.ts`) which will load the HTML file that is the entry point to the app

## Tutorials

https://github.com/davidpet/tutorials/tree/master/Electron/quickstart-app - basics of loading HTML and JS as a desktop app
  - also has a [commit](https://github.com/davidpet/tutorials/commit/54c6fb359a5a15429dccad90b336b6239cf5d714) to show how to convert JS app to use TS